In [1]:
from lmfit import minimize
from lmfit import Parameters

In [2]:
import pyroll.core as pr
import pyroll.pillar_model
import pyroll.neutral_point_estimator
import pyroll.sparling_spreading
import pyroll.hensel_force_torque
import pyroll.freiberg_flow_stress
import pyroll.ring_model_thermal

In [3]:
from in_profile import create_in_profile_from_surface_temperature
from rolling_sequence import roughing_sequence, intermediate_sequence, finishing_sequence_e112, finishing_sequence_e107
from utilities import *

In [4]:
pyroll.pillar_model.Config.PILLAR_COUNT = 30
pyroll.neutral_point_estimator.Config.ESTIMATOR = "EQUAL"
ROLL_SURFACE_TEMPERATURE = 50 + 273.15
DISK_ELEMENT_COUNT = 30

In [5]:
@pr.CoolingPipe.DiskElement.heat_transfer_coefficient
def heat_transfer_coefficient_model_moreales(self: pr.CoolingPipe.DiskElement):
    mean_surface_temperature = (self.in_profile.surface_temperature + self.out_profile.surface_temperature) / 2
    profile_surface_temperature_celsius = mean_surface_temperature - 273.15
    return 13950 + 5.12 * 1e6 / profile_surface_temperature_celsius

In [6]:
in_profile = create_in_profile_from_surface_temperature(1050 + 273.15)
in_profile.ring_temperatures = in_profile.ring_temperatures

In [7]:
roughing_train = roughing_sequence(
    roll_surface_temperature=ROLL_SURFACE_TEMPERATURE,
    disk_element_count=DISK_ELEMENT_COUNT
)

intermediate_train = intermediate_sequence(
    roll_surface_temperature=ROLL_SURFACE_TEMPERATURE,
    disk_element_count=DISK_ELEMENT_COUNT
)

finishing_train_e112 = finishing_sequence_e112(
    roll_surface_temperature=ROLL_SURFACE_TEMPERATURE,
    disk_element_count=DISK_ELEMENT_COUNT
)

finishing_train_e107 = finishing_sequence_e107(
    roll_surface_temperature=ROLL_SURFACE_TEMPERATURE,
    disk_element_count=DISK_ELEMENT_COUNT
)


In [8]:
rolling_train_e112 = pr.PassSequence([
    roughing_train,
    intermediate_train,
    finishing_train_e112
])

rolling_train_e107 = pr.PassSequence([
    roughing_train,
    intermediate_train,
    finishing_train_e107
])

In [9]:
rolling_train_e112.flatten()
rolling_train_e107.flatten()

In [11]:
rolling_train_e112.solve(in_profile)

Solution iteration of TwoRollPass '3' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '7' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '8' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '14' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '18-V' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '23-H' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '18-V' exceeded the maximum iteration count of 100. Continuing anyway.


Profile(
    classifiers={'false_round', 'generic_elongation', 'round', 'symmetric'},
    core_temperature=np.float64(1570.0591539343973),
    cross_section=Polygon(
        area=5.939031536528554e-05,
        height=0.008700000000000006,
        perimeter=0.027369332464370438,
        width=0.008768227336460339,
        ),
    length=5171.222724142008,
    material=['BST500', 'steel'],
    ring_temperatures=array([1570.05915393, 1567.12701904, 1558.34092613, 1543.71864333,
           1523.23735286, 1496.84675482, 1464.74408402, 1428.15944014,
           1390.54359123, 1358.13149   , 1337.87659106]),
    strain=np.float64(0.5607017311212023),
    surface_temperature=np.float64(1337.876591055721),
    t=np.float64(36.875179665279155),
    temperature=np.float64(1425.8613020468924),
    weight=2400,
    width=np.float64(0.008768227336460339),
    )

In [12]:
rolling_train_e107.solve(in_profile)

Solution iteration of TwoRollPass '22-V' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '23-H' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '27-H' exceeded the maximum iteration count of 100. Continuing anyway.


Profile(
    classifiers={'false_round', 'generic_elongation', 'round', 'symmetric'},
    core_temperature=np.float64(1564.0246198859436),
    cross_section=Polygon(
        area=5.8962201229988085e-05,
        height=0.008700000000000006,
        perimeter=0.027243885509781043,
        width=0.008532317323115993,
        ),
    length=5208.733944907869,
    material=['BST500', 'steel'],
    ring_temperatures=array([1564.02461989, 1561.13466478, 1552.47023573, 1538.03228003,
           1517.78168798, 1491.70612299, 1460.17725746, 1424.73026868,
           1388.99526951, 1358.82742219, 1340.17168701]),
    strain=np.float64(0.5201949000088957),
    surface_temperature=np.float64(1340.1716870065247),
    t=np.float64(36.85417441104773),
    temperature=np.float64(1423.7200086555586),
    weight=2400,
    width=np.float64(0.008532317323115993),
    )

In [13]:
def extract_wear_contours_from_measurement(groove_name):
    root_dir = Path.cwd()
    wear_data_dir = root_dir / "wear_data" / groove_name

    labels = []
    tonnages = []
    measured_wear_contours = []

    for file in wear_data_dir.iterdir():
        with open(file, "r") as file:
            data = json.load(file)

        for entry in data["wear_data"]:
            labels.append(entry["stand"])
            tonnages.append(entry["tonnage"])
            x_values = np.array([point["x"] * 1e-3 for point in entry["wear_contour"]])
            y_values = np.array([point["y"] * 1e-3 for point in entry["wear_contour"]])
            x_values_shifted = x_values - max(x_values) / 2
            measured_wear_contour = LineString(list(zip(x_values_shifted, y_values)))
            measured_wear_contours.append(measured_wear_contour)

        return labels, tonnages, measured_wear_contours

In [20]:
def calculate_wear_contour_as_function(roll_pass: pr.RollPass, base_wear_coefficient: float, roll_gap_ratio_coefficient: float, tonnage: float):
    vickers_hardness_c15c = 900

    _pillars_wear_length = pillars_wear_length(roll_pass.roll)
    _pillar_deformation_resistance = pillar_deformation_resistance(roll_pass)

    def wear_coefficient(base_wear_coefficient: float, roll_gap_ratio_coefficent: float):
        mean_cross_section = (roll_pass.in_profile.cross_section.area + 2 * roll_pass.out_profile.cross_section.area) / 3
        roll_gap_ratio =  roll_pass.roll.contact_area / mean_cross_section

        return base_wear_coefficient * roll_gap_ratio ** roll_gap_ratio_coefficent

    _wear_coefficient = wear_coefficient(base_wear_coefficient, roll_gap_ratio_coefficient)
    pillar_wear_depths = _wear_coefficient * _pillar_deformation_resistance * _pillars_wear_length / (
            3 * vickers_hardness_c15c)
    total_pillar_wear_depths = pillar_wear_depths * tonnage / roll_pass.in_profile.weight
    wear_depths_with_groove_contour = roll_pass.roll.groove.local_depth(
        roll_pass.out_profile.pillars) + total_pillar_wear_depths
    right_side = list(zip(roll_pass.out_profile.pillars, wear_depths_with_groove_contour))
    left_side = list(zip(-roll_pass.out_profile.pillars[::-1], wear_depths_with_groove_contour[::-1]))
    combined_contour_list = left_side + right_side
    wear_contour = LineString(combined_contour_list)

    return wear_contour

In [21]:
def solve_total_wear_depths_as_contour_line(params):
    sequences = [
        {
            "label": "e11_2",
            "sequence": rolling_train_e112
        },
        {
            "label": "e10_7",
            "sequence": rolling_train_e107
        }
    ]

    area_errors = []
    base_wear_coefficient = params['base_wear_coefficient'].value
    roll_gap_ratio_coefficient = params['roll_gap_ratio_coefficient'].value

    for dict in sequences:
        roll_passes = dict["sequence"].roll_passes
        for roll_pass in roll_passes:
            labels, tonnages, measured_wear_contours = extract_wear_contours_from_measurement(dict["label"])
            for label, tonnage, measured_wear_contour in zip(labels, tonnages, measured_wear_contours):
                if roll_pass.label == label:
                    calculated_wear_contour = calculate_wear_contour_as_function(roll_pass, base_wear_coefficient,roll_gap_ratio_coefficient, tonnage * 1000)
                    area_error = calculate_area_between_contours(calculated_wear_contour, measured_wear_contour)

                    area_errors.append(area_error)
                else:
                    continue
    print(area_errors)
    rms_error = root_mean_square_value(np.array(area_errors))
    print("RMS Error: ", rms_error)
    return rms_error


In [22]:
params = Parameters()
params.add('base_wear_coefficient', value=1e-6, min=1e-12, max=1e-3, vary=True)
params.add('roll_gap_ratio_coefficient', value=1e-6, min=1e-12, max=1e-3, vary=True)

In [23]:
result = minimize(
    solve_total_wear_depths_as_contour_line,
    params,
    nan_policy='omit',
    max_nfev=100,
    #method="least_squares"
    method="cobyla"
)

[np.float64(0.01686005929270209), np.float64(0.01024216355080671), np.float64(0.047391729264827966), np.float64(0.010852154276129174), np.float64(0.02330441596877932), np.float64(0.009021442531524324), np.float64(0.01700616657052846), np.float64(0.008283575190405859), np.float64(0.008371959865682094), np.float64(0.004857607347681181), np.float64(0.009904051762089701), np.float64(0.008959128374927598), np.float64(0.0058469818269668156), np.float64(0.007287554287511434), np.float64(0.0035969188829024375), np.float64(0.003872220907337281)]
RMS Error:  0.016051651326496857
[np.float64(4.333164302112931), np.float64(2.632325303635149), np.float64(12.18060921307842), np.float64(2.788894145507086), np.float64(5.989624722515311), np.float64(2.318676612593277), np.float64(4.3711239190860125), np.float64(2.128890948845632), np.float64(2.1655968012997655), np.float64(1.2510194514929955), np.float64(2.55539596117806), np.float64(2.304887210967698), np.float64(1.5087243941357076), np.float64(1.8740

In [24]:
result.params.pretty_print()

Name                           Value      Min      Max   Stderr     Vary     Expr Brute_Step
base_wear_coefficient       3.106e-10    1e-12    0.001     None     True     None     None
roll_gap_ratio_coefficient  8.222e-07    1e-12    0.001     None     True     None     None


In [25]:
result.residual

array([2.17389944e-05])

In [26]:
from lmfit import fit_report

In [27]:
print(fit_report(result))

[[Fit Statistics]]
    # fitting method   = COBYLA
    # function evals   = 31
    # data points      = 1
    # variables        = 2
    chi-square         = 4.7258e-10
    reduced chi-square = 4.7258e-10
    Akaike info crit   = -17.4728059
    Bayesian info crit = -21.4728059
##  Warning: uncertainties could not be estimated:
    this fitting method does not natively calculate uncertainties
    and numdifftools is not installed for lmfit to do this. Use
    `pip install numdifftools` for lmfit to estimate uncertainties
    with this fitting method.
[[Variables]]
    base_wear_coefficient:       3.1064e-10 (init = 1e-06)
    roll_gap_ratio_coefficient:  8.2225e-07 (init = 1e-06)
